# Capstone week 3 part1
## Scrape wikipedia data and generate dataframe
### Since the wiki data followed simple pattern decided to use bash script

In [2]:
!curl https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M -o wiki.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 52234  100 52234    0     0   228k      0 --:--:-- --:--:-- --:--:--  228k


1. remove newline character
2. grep the pattern and only print matched
3. from the pattern get the data and store in csv

In [3]:
!echo "PostalCode:Borough:Neighborhood" > loc.csv
!cat wiki.txt |\
tr -d \\n |\
grep -Pzo "<tr><td>.*?(</td>)<td>.*?(</td>)<td>.*?(</td>)?(</tr>)" |\
sed -n "s|<tr><td>\(.*\)</td><td>\(.*\)</td><td>\(.*\)</td></tr>|\1:\2:\3|p" >> loc.csv

### load the data in pandas dataframe

In [4]:
import pandas as pd
pc_locs_df = pd.read_csv('loc.csv', delimiter=':')
pc_locs_df = pc_locs_df[pc_locs_df['Borough'].str.lower() != 'not assigned'].reset_index().drop('index', axis=1)

In [5]:
pc_locs_df['Neighborhood'] = pc_locs_df.apply(axis=1, func=lambda x:x[2] if x[2].strip() else x[1])

In [6]:
pc_locs_df.shape

(103, 3)

### could not find cordinates in geopy so using csv from internet for lat, lon

In [7]:
cords = pd.read_csv('http://cocl.us/Geospatial_data').set_index('Postal Code')
def f(x):
    return cords.loc[x[0]]
pc_locs_df[['latitude', 'longitude']] = pc_locs_df.apply(axis=1, func=f)

In [8]:
pc_locs_df.head()

,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [9]:
pc_locs_df.to_csv('loc.csv')
pc_locs_df.shape

(103, 5)